In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
#import data from files

x_header = pd.read_csv("x_header.csv")
train_data = pd.read_csv('train_data.csv', header = None)
test_data = pd.read_csv('test_data.csv', header = None)
train_labels = pd.read_csv('train_labels.csv', header = None, dtype = 'category' )


#create variables 
header = pd.DataFrame(data = x_header)
X = pd.DataFrame(data = train_data)
X.columns = header.ID
X_test = pd.DataFrame(data = test_data)
X_test.columns = header.ID
y = pd.DataFrame(data = train_labels)
y.columns = ['Genre']

display(HTML(X.tail(5).to_html(max_rows=5)))


ID,Rhythm_Mean_1,Rhythm_Mean_2,Rhythm_Mean_3,Rhythm_Mean_4,Rhythm_Mean_5,Rhythm_Mean_6,Rhythm_Mean_7,Rhythm_Mean_8,Rhythm_Mean_9,Rhythm_Mean_10,Rhythm_Mean_11,Rhythm_Mean_12,Rhythm_Mean_13,Rhythm_Mean_14,Rhythm_Mean_15,Rhythm_Mean_16,Rhythm_Mean_17,Rhythm_Mean_18,Rhythm_Mean_19,Rhythm_Mean_20,Rhythm_Mean_21,Rhythm_Mean_22,Rhythm_Mean_23,Rhythm_Mean_24,Rhythm_Median_1,Rhythm_Median_2,Rhythm_Median_3,Rhythm_Median_4,Rhythm_Median_5,Rhythm_Median_6,Rhythm_Median_7,Rhythm_Median_8,Rhythm_Median_9,Rhythm_Median_10,Rhythm_Median_11,Rhythm_Median_12,Rhythm_Median_13,Rhythm_Median_14,Rhythm_Median_15,Rhythm_Median_16,Rhythm_Median_17,Rhythm_Median_18,Rhythm_Median_19,Rhythm_Median_20,Rhythm_Median_21,Rhythm_Median_22,Rhythm_Median_23,Rhythm_Median_24,Rhythm_Var_1,Rhythm_Var_2,Rhythm_Var_3,Rhythm_Var_4,Rhythm_Var_5,Rhythm_Var_6,Rhythm_Var_7,Rhythm_Var_8,Rhythm_Var_9,Rhythm_Var_10,Rhythm_Var_11,Rhythm_Var_12,Rhythm_Var_13,Rhythm_Var_14,Rhythm_Var_15,Rhythm_Var_16,Rhythm_Var_17,Rhythm_Var_18,Rhythm_Var_19,Rhythm_Var_20,Rhythm_Var_21,Rhythm_Var_22,Rhythm_Var_23,Rhythm_Var_24,Rhythm_Kurtosis_1,Rhythm_Kurtosis_2,Rhythm_Kurtosis_3,Rhythm_Kurtosis_4,Rhythm_Kurtosis_5,Rhythm_Kurtosis_6,Rhythm_Kurtosis_7,Rhythm_Kurtosis_8,Rhythm_Kurtosis_9,Rhythm_Kurtosis_10,Rhythm_Kurtosis_11,Rhythm_Kurtosis_12,Rhythm_Kurtosis_13,Rhythm_Kurtosis_14,Rhythm_Kurtosis_15,Rhythm_Kurtosis_16,Rhythm_Kurtosis_17,Rhythm_Kurtosis_18,Rhythm_Kurtosis_19,Rhythm_Kurtosis_20,Rhythm_Kurtosis_21,Rhythm_Kurtosis_22,Rhythm_Kurtosis_23,Rhythm_Kurtosis_24,Rhythm_Skewness_1,Rhythm_Skewness_2,Rhythm_Skewness_3,Rhythm_Skewness_4,Rhythm_Skewness_5,Rhythm_Skewness_6,Rhythm_Skewness_7,Rhythm_Skewness_8,Rhythm_Skewness_9,Rhythm_Skewness_10,Rhythm_Skewness_11,Rhythm_Skewness_12,Rhythm_Skewness_13,Rhythm_Skewness_14,Rhythm_Skewness_15,Rhythm_Skewness_16,Rhythm_Skewness_17,Rhythm_Skewness_18,Rhythm_Skewness_19,Rhythm_Skewness_20,Rhythm_Skewness_21,Rhythm_Skewness_22,Rhythm_Skewness_23,Rhythm_Skewness_24,Rhythm_Min_1,Rhythm_Min_2,Rhythm_Min_3,Rhythm_Min_4,Rhythm_Min_5,Rhythm_Min_6,Rhythm_Min_7,Rhythm_Min_8,Rhythm_Min_9,Rhythm_Min_10,Rhythm_Min_11,Rhythm_Min_12,Rhythm_Min_13,Rhythm_Min_14,Rhythm_Min_15,Rhythm_Min_16,Rhythm_Min_17,Rhythm_Min_18,Rhythm_Min_19,Rhythm_Min_20,Rhythm_Min_21,Rhythm_Min_22,Rhythm_Min_23,Rhythm_Min_24,Rhythm_Max_1,Rhythm_Max_2,Rhythm_Max_3,Rhythm_Max_4,Rhythm_Max_5,Rhythm_Max_6,Rhythm_Max_7,Rhythm_Max_8,Rhythm_Max_9,Rhythm_Max_10,Rhythm_Max_11,Rhythm_Max_12,Rhythm_Max_13,Rhythm_Max_14,Rhythm_Max_15,Rhythm_Max_16,Rhythm_Max_17,Rhythm_Max_18,Rhythm_Max_19,Rhythm_Max_20,Rhythm_Max_21,Rhythm_Max_22,Rhythm_Max_23,Rhythm_Max_24,Chroma_Mean_1,Chroma_Mean_2,Chroma_Mean_3,Chroma_Mean_4,Chroma_Mean_5,Chroma_Mean_6,Chroma_Mean_7,Chroma_Mean_8,Chroma_Mean_9,Chroma_Mean_10,Chroma_Mean_11,Chroma_Mean_12,Chroma_Std_1,Chroma_Std_2,Chroma_Std_3,Chroma_Std_4,Chroma_Std_5,Chroma_Std_6,Chroma_Std_7,Chroma_Std_8,Chroma_Std_9,Chroma_Std_10,Chroma_Std_11,Chroma_Std_12,Chroma_Min_1,Chroma_Min_2,Chroma_Min_3,Chroma_Min_4,Chroma_Min_5,Chroma_Min_6,Chroma_Min_7,Chroma_Min_8,Chroma_Min_9,Chroma_Min_10,Chroma_Min_11,Chroma_Min_12,Chroma_Max_1,Chroma_Max_2,Chroma_Max_3,Chroma_Max_4,Chroma_Max_5,Chroma_Max_6,Chroma_Max_7,Chroma_Max_8,Chroma_Max_9,Chroma_Max_10,Chroma_Max_11,Chroma_Max_12,MFCC_Mean_1,MFCC_Mean_2,MFCC_Mean_3,MFCC_Mean_4,MFCC_Mean_5,MFCC_Mean_6,MFCC_Mean_7,MFCC_Mean_8,MFCC_Mean_9,MFCC_Mean_10,MFCC_Mean_11,MFCC_Mean_12,MFCC_Std_1,MFCC_Std_2,MFCC_Std_3,MFCC_Std_4,MFCC_Std_5,MFCC_Std_6,MFCC_Std_7,MFCC_Std_8,MFCC_Std_9,MFCC_Std_10,MFCC_Std_11,MFCC_Std_12,MFCC_Min_1,MFCC_Min_2,MFCC_Min_3,MFCC_Min_4,MFCC_Min_5,MFCC_Min_6,MFCC_Min_7,MFCC_Min_8,MFCC_Min_9,MFCC_Min_10,MFCC_Min_11,MFCC_Min_12,MFCC_Max_1,MFCC_Max_2,MFCC_Max_3,MFCC_Max_4,MFCC_Max_5,MFCC_Max_6,MFCC_Max_7,MFCC_Max_8,MFCC_Max_9,MFCC_Max_10,MFCC_Max_11,MFCC_Max_12
4358,3050.6,6570.7,5138.4,3620.7,3059.7,3909.8,3408.6,2232.5,2344.70,2509.10,2230.5,2107.7,3513.3,4666.2,5935.8,6765.4,6415.1,4797.3,4324.90,2810.70,1845.20,1406.10,703.140

In [2]:
#append x_y_train
#x_y_train = x_train.join(y_train)

#View value frequencies from each columns 
#for i in x_y_train.columns:
    #print("Value count", i, ": ", x_y_train[i].value_counts().head(5))
#Drop bad data
X = X.drop(columns = ['MFCC_Mean_1','MFCC_Mean_2','MFCC_Mean_3','MFCC_Mean_4','MFCC_Min_1'])
X_test = X_test.drop(columns = ['MFCC_Mean_1','MFCC_Mean_2','MFCC_Mean_3','MFCC_Mean_4','MFCC_Min_1'])

X.columns



Index(['Rhythm_Mean_1', 'Rhythm_Mean_2', 'Rhythm_Mean_3', 'Rhythm_Mean_4',
       'Rhythm_Mean_5', 'Rhythm_Mean_6', 'Rhythm_Mean_7', 'Rhythm_Mean_8',
       'Rhythm_Mean_9', 'Rhythm_Mean_10',
       ...
       'MFCC_Max_3', 'MFCC_Max_4', 'MFCC_Max_5', 'MFCC_Max_6', 'MFCC_Max_7',
       'MFCC_Max_8', 'MFCC_Max_9', 'MFCC_Max_10', 'MFCC_Max_11',
       'MFCC_Max_12'],
      dtype='object', name='ID', length=259)

In [3]:
#Normalize data according to group
group = ((0,24),(24,48),(48,72),(72,96),(96,120),(120,144),(144,168),(216,224),(224,236),(236,247),(247,259))

def normalization(X,group):
    for i,j in group:
        maximum = np.amax(np.amax(X.iloc[:,i:j]))
        minimum = np.amin(np.amin(X.iloc[:,i:j]))
        dfnorm = np.divide(np.subtract(X.iloc[:,i:j],minimum),(maximum - minimum))
        X.iloc[:,i:j] = dfnorm
    return X

X = normalization(X,group)
X_test = normalization(X_test,group)


#display(HTML(x_y_train.head(15).to_html(max_rows=15)))


In [8]:
#Split data to test-validation set
y = np.ravel(y)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

#Apply logistic regression
logreg = LogisticRegression(penalty='l2', solver='lbfgs', max_iter = 500, multi_class='multinomial')
logreg.fit(X_train, y_train)

print(logreg.score(X_train,y_train))
y_val_pred = logreg.predict(X_val)
print(logreg.score(X_val,y_val))

#Apply cross-validation
from sklearn.model_selection import cross_validate
from sklearn import metrics

#Test result from different k
#for i in range(2,6):
    #scores = cross_validate(logreg, X_train, y_train, cv=i)
    #print("Cross-validated scores:", scores["test_score"])

scores = cross_validate(logreg, X_train, y_train, cv=3)
print("Cross-validated scores:", scores["test_score"])


0.650017105713
0.629861111111
Cross-validated scores: [ 0.61617195  0.60102564  0.63542739]


In [6]:
y_test_pred = logreg.predict(X_test)
indx = X_test.index
result = pd.DataFrame(index = indx, columns = ["Sample_id","Sample_label"])
result["Sample_id"] = result.index+1
result["Sample_label"] = y_test_pred
result.to_csv("solution.csv", index = False)
